# EDA Report of e-commerce data

In [ ]:
!pip install pandas json5

In [ ]:
import pandas as pd
import json
from datetime import datetime


In [7]:
df= pd.read_csv("E-commerce.csv")
df.head()

,Customer ID,Age,Gender,Location,Annual Income,Purchase History,Browsing History,Product Reviews,Time on Site
0,1001,25,Female,City D,45000,"[{""Date"": ""2022-03-05"", ""Category"": ""Clothing""...","[{""Timestamp"": ""2022-03-10T14:30:00Z""}, {""Time...","Great pair of jeans, very comfortable. Rating:...",32.50
1,1001,28,Female,City D,52000,"[{""Product Category"": ""Clothing"", ""Purchase Da...","[{""Product Category"": ""Home & Garden"", ""Timest...",Great customer service!,123.45
2,1001,28,Female,City D,65000,"[{""Product Category"": ""Electronics"", ""Purchase...","[{""Product Category"": ""Clothing"", ""Timestamp"":...",Great electronics. The sound quality is excell...,125.60
3,1001,45,Female,City D,70000,"{'Purchase Date': '2022-08-15', 'Product Categ...",{'Timestamp': '2022-09-03 14:30:00'},"{""Product 1"": {""Rating"": 4, ""Review"": ""Great e...",327.60
4,1002,34,Male,City E,45000,"{'Purchase Date': '2022-07-25', 'Product Categ...",{'Timestamp': '2022-08-10 17:15:00'},"{""Product 1"": {""Rating"": 3, ""Review"": ""Good pr...",214.90


In [11]:
def clean_json(value):
    """Convert JSON-like strings into Python objects."""
    if isinstance(value, str):  # If value is a string
        try:
            value = value.replace("'", '"')  # Fix incorrect quotes
            return json.loads(value)  # Convert to dictionary/list
        except:
            return []  # Return empty list if conversion fails
    return value if isinstance(value, list) else []  # Ensure list format


In [12]:
# Apply JSON cleaning to relevant columns
df["Purchase History"] = df["Purchase History"].fillna("[]").apply(clean_json)


In [13]:
# Normalize Purchase History into structured rows
purchase_records = []
for _, row in df.iterrows():
    for purchase in row["Purchase History"]:
        if isinstance(purchase, dict):  # Ensure it's a dictionary
            purchase_records.append({
                "Customer_ID": row["Customer ID"],
                "Age": row["Age"],
                "Gender": row["Gender"],
                "Location": row["Location"],
                "Annual_Income": row["Annual Income"],
                "Category": purchase.get("Category") or purchase.get("Product Category", "Unknown"),  # Handle variations
                "Price": purchase.get("Price", 0),
                "Purchase_Date": purchase.get("Date") or purchase.get("Purchase Date", "Unknown"),
                "Last_Updated": datetime.now().strftime("%Y-%m-%d %H:%M:%S")  # Add timestamp
            })


In [14]:
df_cleaned = pd.DataFrame(purchase_records)
df_cleaned.head()

,Customer_ID,Age,Gender,Location,Annual_Income,Category,Price,Purchase_Date,Last_Updated
0,1001,25,Female,City D,45000,Clothing,34.99,2022-03-05,2025-03-16 01:25:54
1,1001,25,Female,City D,45000,Electronics,129.99,2022-02-12,2025-03-16 01:25:54
2,1001,25,Female,City D,45000,Home & Garden,29.99,2022-01-20,2025-03-16 01:25:54
3,1001,28,Female,City D,52000,Clothing,34.56,2022-05-15,2025-03-16 01:25:54
4,1001,28,Female,City D,52000,Electronics,150.99,2022-06-02,2025-03-16 01:25:54


In [15]:
output_file = "cleaned_ecommerce_data.csv"
df_cleaned.to_csv(output_file, index=False)

print(f"✅ Processed dataset saved as {output_file}")

✅ Processed dataset saved as cleaned_ecommerce_data.csv
